In [ ]:
import matplotlib.pyplot as plt
import mpl_lego as mplego
import numpy as np
import pandas as pd
import pickle

from pyprojroot import here
from hate_target import keys
from sklearn.metrics import roc_auc_score, average_precision_score
from mpl_lego.labels import bold_text

%matplotlib inline

In [ ]:
mplego.style.use_latex_style()

In [ ]:
sorted_labels = sorted(keys.target_labels)

In [ ]:
# Import dataset
comment_id = 'comment_id'
text_col = 'predict_text'
threshold = 0.5

# Read in data
data_path = "~/data/hatespeech/unfiltered_ratings.feather"
data = pd.read_feather(data_path)
comments = data[[comment_id, text_col]].drop_duplicates().sort_values(comment_id)
# Determine target identities
agreement = data[[comment_id] + keys.target_groups].groupby(comment_id).agg('mean')
is_target = (agreement >= threshold).astype('int').reset_index(level=0).merge(right=comments, how='left')
# Extract data for training models
x = is_target[text_col].values
identities = agreement[sorted(keys.target_groups)]
y = [identities[col].values.astype('int')[..., np.newaxis] for col in identities]

n_groups = len(sorted_labels)

In [ ]:
exp_path = here('experiments/exp1_use_v4_h64_b32_d4.pkl')

In [ ]:
with open(exp_path, 'rb') as file:
    n_epochs, train_idxs, test_idxs, test_predictions, test_scores, chance = pickle.load(file)

In [ ]:
test_accs = test_scores[:, 9:]

In [ ]:
acc_over_chance = test_accs / chance
avg_acc_over_chance = np.mean(acc_over_chance, axis=0)

In [ ]:
log_odds_diff = np.log(test_accs / (1 - test_accs)) - np.log(chance / (1 - chance))
avg_log_odds_diff = np.mean(log_odds_diff, axis=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

axes[0].barh(-np.arange(n_groups), avg_acc_over_chance)
axes[1].barh(-np.arange(n_groups), avg_log_odds_diff)

axes[0].set_xlim([1.0, 1.05])
for ax in axes:
    ax.tick_params(labelsize=15)
    ax.set_yticks(-np.arange(n_groups))
    ax.set_yticklabels(bold_text(sorted_labels))
    
axes[0].set_xlabel(bold_text('Accuracy / Chance'), fontsize=17)
axes[1].set_xlabel(bold_text('Log-Odds Difference'), fontsize=17)
plt.savefig('accuracies.pdf', bbox_inches='tight')

In [ ]:
auc_rocs = np.array([roc_auc_score(y[i].ravel(), test_predictions[i].ravel()) for i in range(n_groups)])
auc_prcs = np.array([average_precision_score(y[i].ravel(), test_predictions[i].ravel()) for i in range(8)])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

axes[0].barh(-np.arange(n_groups), auc_rocs)
axes[1].barh(-np.arange(n_groups), auc_prcs)

axes[0].set_xlim([0.8, 1.0])
axes[1].set_xlim([0, 0.35])

for ax in axes:
    ax.tick_params(labelsize=15)
    ax.set_yticks(-np.arange(n_groups))
    ax.set_yticklabels(bold_text(sorted_labels))
    
axes[0].set_xlabel(bold_text('ROC-AUC'), fontsize=17)
axes[1].set_xlabel(bold_text('PR-AUC'), fontsize=17)

plt.savefig('aucs.pdf', bbox_inches='tight')